# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на 4 вопроса:
 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 4. Как разные цели кредита влияют на его возврат в срок?
 
 **Ход исследования**
 1. Определить и заполнить пропущенные значения:
    - описать, какие пропущенные значения я обнаружил;
    - привести возможные причины появления пропусков в данных;
    - объяснить, по какому принципу заполнены пропуски;
 2. Заменить вещественный тип данных на целочисленный:
    - пояснить, как выбирали метод для изменения типа данных;
 3. Удалить дубликаты:
    - пояснить, как выбирал метод для поиска и удаления дубликатов в данных;
    - привести возможные причины появления дубликатов;
 4. Выделить леммы в значениях столбца с целями получения кредита:
    - описать, как я проводил лемматизацию целей кредита;
 5. Категоризировать данные:
    - перечислить, какие «словари» я выделил для этого набора данных, и объяснить, почему.
 

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd
from pymystem3 import Mystem
df = pd.read_csv('data.csv')#/datasets/data.csv
display(df.info())
#display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

**Вывод**

В таблице семь столбцов. Тип данных разный, возможно он в каких-то столбцах не оптимален, но разберемся с этим в предобработке.

Согласно документации к данным:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Подозрительно одинаковое количество пропусков двух столбцах, скорее всего нет данных по этим людям, посмотрим дальше. 

В каждой строке таблицы — данные о клиенте банка. Это статистика о платёжеспособности клиентов, где часть колонок описывает клиента: пол, возраст, семейное положение, количество детей, уровень образования. Остальные данные рассказывают об экономических особенностях пользователя: тип занятости, доход,имеются ли задолженности.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(df['days_employed'].unique())
print(df['total_income'].unique())
print(df['children'].value_counts())
df['children'] = df['children'].abs()#в столбце оказались отрицательные значения, перевел их в положительные
print(df['gender'].value_counts())

[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
[253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [3]:
df.loc[df['children'] == 20, 'children'] = 2 #меняю 20 на 2, так как похоже это опечатка
print(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [4]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'F' #это тоже мне кажется опечаткой, меняю на самое встречающееся значение
print(df['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64


Как мы видим, в стобцах с пропущенными значениями есть только числа, но в столбце 'days_employed' они бывают аномально большие и отрицательные. Заменим их таким образом, предполагая, что очень большие значения даны в часах, а отрицательные возвращаем положительными, так как стаж не мжет быть отрицательным: 

In [5]:
df['days_employed'] = df['days_employed'].apply(abs)
df.loc[df['days_employed'] >30000, 'days_employed'] = df.loc[df['days_employed'] >30000, 'days_employed'] / 24
#а теперь заполняем пропуски медианами, в столбце total_income преподаватель сказал,
#что все норм и к значениям придираться не надо, мол это учебные значения
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median()
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median()
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

**Вывод**

Таким образом, заполнили все пропуски в данных. Почему произошли такие ошибки? Скорее всего, в данных встречается информация из двух систем. Одна из них считает стаж в часах, а другая наоборот. Для заполнения пропущенных значений использовал медиану, потому что это более объективная информация о зарплате сотрудников, чем среднее значение. То же самое и со стажем, там есть очень большие значения стажа, поэтому медиана более корректно заменит пропуск и меньше скажется на всей статистике.

### Замена типа данных

In [6]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


None

**Вывод**

Так как у нас вещественный тип данных в столбцах со стажем и с доходом в месяц, можно применить к этим столбцам метод astype(), потому что он может перевести из вещественного типа в целочисленный, а эти данные удобнее и красивее представлять целочисленно, значения после запятой не будут нести большого значения, так как значения в этих слобцах "продолжительные".

### Обработка дубликатов

In [7]:
#df.duplicated()
df['education'] = df['education'].str.lower()#примводим все строки в стобце с образованием к нижнеему регистру
#display(df.head(30))
#находим полные дубли
#df_dubl = df[df.duplicated(keep = False)] # keep = False выведет все совпадения
#display(df_dubl)
print(df.duplicated().sum())#количество добликатов до их удаления
df=df.drop_duplicates()
print(df.duplicated().sum())#и после их удаления, преподаватель сказал, что в проекте смотрим только полные дубли строк


71
0


**Вывод**

Для поиска дубликатов использовал метод duplicated(), так как нужно было отыскать полные дубликаты строк. Дубликаты могли появится из-за разного регистра в столбце "образование", могли образоваться при соединении нескольких таблиц в одну, этот клиент проходить в системе под разными айди.

### Лемматизация

In [8]:

#from pymystem3 import Mystem
m = Mystem()

lemmas = m.lemmatize(" ".join(df['purpose'].unique()))

Дальше посмотрим, какие леммы встрачаются чаще всего.

In [9]:
from collections import Counter
Counter(lemmas).most_common() 

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

Выделяем такие категории, как ['недвижимость', 'автомобиль', 'образование', 'жилье', 'операция']. Покупку не вношу, потому что она как раз входит внутрь других категорий, она не несет смысловой нагрузки, клиенту банка нужны деньги на покупку(ясное дело), а вот на покупку чего это нас и интересует.

In [10]:
#тут две функции, потому что я проверял, пофиг ли ядру на моей локальной машине на сложность кода, 
#оказалось пофиг и пришлось делать в тренажере
#from pymystem3 import Mystem
m = Mystem()
common_purposes = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'операция']
 
def purpose_category(value):
    lemmas = m.lemmatize(value)
    for item in common_purposes:
        if item in lemmas:
            return item
    return 'прочее'
#def purpose_category(cell_value):
#    cell_lemmas = m.lemmatize(cell_value)
#    if 'недвижимость' in cell_lemmas:
#        return 'недвижимость'
#    if 'автомобиль' in cell_lemmas:
#        return 'авто'
#    if 'образование' in cell_lemmas:
#        return 'образование'
#    if 'жилье' in cell_lemmas:
#        return 'жилье'
#    if 'операция' in cell_lemmas:
#        return 'операция'
#    return 'прочее'
 
df['purpose_category'] = df['purpose'].apply(purpose_category)
display(df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,прочее
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,прочее
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [11]:
#df[df['purpose_category'] == 'операция']#'операция'
df['purpose_category'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
прочее          2324
Name: purpose_category, dtype: int64

**Вывод**

Мы проделали немалый путь, а именно:
- Получили уникальные значения purpose, склеили их в одну строку с помощью join и лемматизировали.
- С помощь Counter посчитали частоту появления слов.
- После этого выделили (вручную) из этого списка самые частые существительные - основные категории. 
- Написали функцию для apply, которая лематизирует последовательно каждую ячейку столбца purpose и проверяет, какая из категорий обнаруживается в лемматизированном варианте.

Таким образом, получилось пять катеорий.

### Категоризация данных

Здесь выделим словари, которые помогут ответить на вопросы исследования.

In [12]:
slovar_education = df[['education_id', 'education']].drop_duplicates().set_index('education_id')#словарь для категорий по образованию
display(slovar_education)

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [13]:
slovar_family = df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')#словарь для категорий по семейному положению
display(slovar_family)

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


In [14]:
def children_group(value):#создал две группы, потому что спрашивают наличие ребенка
   
    if value == 0:
        return 'нет'
    return 'есть' 

df.insert(1, 'children_id', df['children'].apply(children_group))
display(df.head(10))

,children,children_id,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,есть,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,есть,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,нет,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,есть,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,нет,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,прочее
5,0,нет,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,нет,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,нет,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,есть,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,прочее
9,0,нет,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [15]:
slovar_children = df[['children_id', 'children']].drop_duplicates().set_index('children_id')#словарь для категорий по наличию детей
display(slovar_children)#не знаю пригодится ли, пусть пока будет

,children
children_id,
есть,1
нет,0
есть,3
есть,2
есть,4
есть,5


In [16]:
df_children = df[['children_id', 'children']]
display(df_children.groupby('children').count())#просто посмотреть сколько людей в каждой категории

,children_id
children,
0,14091
1,4855
2,2128
3,330
4,41
5,9


In [17]:
print(df['total_income'].min())
print(df['total_income'].max())#посмотрел минимум и максимум чтобы сориентироваться по группам
print(df['total_income'].mean())
print(df['total_income'].median())#медиана и среднее ничего мне не сказали

20667
2265604
165225.6006339144
145017.0


In [18]:
def total_income_group(value):#решил, что поделю на 5 групп, как и везде 
   
    if value <= 70000:
        return 'до 70к'
    if value <= 200000:
        return 'от 70к до 200к'
    if value <= 500000:
        return 'от 200к до 500к'
    if value <= 1000000:
        return 'от 500к до 1м'
    return 'свыше 1м' 

df.insert(12, 'total_income_id', df['total_income'].apply(total_income_group))
display(df.head(15))


,children,children_id,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_id,purpose,purpose_category
0,1,есть,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,от 200к до 500к,покупка жилья,жилье
1,1,есть,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,от 70к до 200к,приобретение автомобиля,автомобиль
2,0,нет,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,от 70к до 200к,покупка жилья,жилье
3,3,есть,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,от 200к до 500к,дополнительное образование,образование
4,0,нет,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,от 70к до 200к,сыграть свадьбу,прочее
5,0,нет,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,от 200к до 500к,покупка жилья,жилье
6,0,нет,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,от 200к до 500к,операции с жильем,жилье
7,0,нет,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,от 70к до 200к,образование,образование
8,2,есть,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,от 70к до 200к,на проведение свадьбы,прочее
9,0,нет,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,от 70к до 200к,покупка жилья для семьи,жилье


In [19]:
df_income = df[['total_income_id', 'total_income']]
display(df_income.groupby('total_income_id').count())#посчитал, сколько вхождений в каждую категорию,
#чтобы прикинуть группы вообще нормальны вышли или нет

,total_income
total_income_id,
до 70к,1474
от 200к до 500к,4844
от 500к до 1м,197
от 70к до 200к,14914
свыше 1м,25


**Вывод**

Выделил словари, которые нужны для ответов на вопросы, а именно словарь образований, словарь семейного статуса, словарь наличия детей и словарь для уровня дохода.

## Шаг 3. Ответьте на вопросы

Перед проверкой вопросов мне нужен еще один столбец с отсутствием задолженности

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
report = df.pivot_table(index='children_id', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)

,no_debt,debt,%
children_id,,,
есть,6685,678,9.208203
нет,13028,1063,7.543822


**Вывод**

У клиентов с детьми немного выше процент невозврата кредита, почти 2 процента. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
report = df.pivot_table(index='family_status', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**

У людей неженатых и находящихся в гражданском браке чуть выше процент задолженности, чем у всех остальных, самый низкий процент у вдовцов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
report = df.pivot_table(index='total_income_id', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)

,no_debt,debt,%
total_income_id,,,
до 70к,1373,101,6.852103
от 200к до 500к,4500,344,7.101569
от 500к до 1м,185,12,6.091371
от 70к до 200к,13632,1282,8.595950
свыше 1м,23,2,8.000000


**Вывод**

По доходу тоже почти все одинаково и в пределах двух процентов, лучше возвращают кредиты люди с доходом от 500к до 1 миллиона, а хуже всех с доходом 70к до 200к(рублей скорее всего, вроде неизвестны единицы измерения).

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
report = df.pivot_table(index='purpose_category', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)

,no_debt,debt,%
purpose_category,,,
автомобиль,3903,403,9.359034
жилье,4152,308,6.905830
недвижимость,5877,474,7.463392
образование,3643,370,9.220035
прочее,2138,186,8.003442


**Вывод**

И здесь мы не видим сильно явных должников, немного хуже возвращаются кредиты взятые на приобретение автомобиля и на образование, а лучше возвращаются кредиты взятые на недвижимость. Мне кажется, что это незначительные отклонения, возможно бизнес посчитает иначе. Преподаватель сказал, что узнаем про z-тест для пропорций и сможем определить статистически значимы ли такие отклонения.

Решил проверить по гендеру еще.

In [24]:
report = df.pivot_table(index='gender', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)
#gender

,no_debt,debt,%
gender,,,
F,13181,994,7.012346
M,6532,747,10.262399


И по образованию.

In [25]:
report = df.pivot_table(index='education', columns='debt', values='total_income', aggfunc='count')
report.columns = ['no_debt', 'debt']
report['%'] = report['debt']/(report['no_debt'] + report['debt'])*100
report = report.fillna(0)
display(report)
#education

,no_debt,debt,%
education,,,
высшее,4972.0,278.0,5.295238
начальное,251.0,31.0,10.992908
неоконченное высшее,676.0,68.0,9.139785
среднее,13808.0,1364.0,8.990245
ученая степень,6.0,0.0,0.000000


## Шаг 4. Общий вывод

Мы попытались ответить на 4 вопроса и дополнительно я задал еще два, по обязательным вопросам мы определили такие закономерности:
- У клиентов с детьми немного выше процент невозврата кредита, почти 2 процента.
- У людей неженатых и находящихся в гражданском браке чуть выше процент задолженности, чем у всех остальных. Самый низкий процент у вдовцов.
- Лучше возвращают кредиты люди с доходом от 500к до 1 миллиона, а хуже всех с доходом 70к до 200к(рублей скорее всего, вроде бы неизвестны единицы измерения).
- Немного хуже возвращаются кредиты взятые на приобретение автомобиля и на образование, а лучше возвращаются кредиты взятые на недвижимость.

На мои доп.вопросы:
- Женщины лучше возвращают кредиты, чем мужчины.
- Чем выше уровень образованности заещика, тем вероятнее он вернет кредит. Исключение состовляют люди с неоконченным высшим образованием, у них процент задолженностей выше, чем у людей со средним уровнем образования.